<a href="https://colab.research.google.com/github/sriyamistry0-lang/CRISPR_Colours/blob/main/Off_Target_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- IMPORTS ---
try:
    import Bio
except:
    !pip install biopython

from Bio import SeqIO       # For reading real genome files (FASTA/GenBank)
from Bio.Seq import Seq     # For handling DNA strings
import pandas as pd         # For organizing off-target results into a table

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.0 MB/s eta 0:00:00


In [6]:
import datetime

# 1. THE FUNCTION DEFINITION
def scan_off_targets_optimized(spacer, genome_to_check, name="Unknown"):
    spacer = spacer.upper().strip() # Clean the input
    print(f"\n--- 🛡️ Starting Scan: {name} ---")
    print(f"Targeting: {spacer}")

    spacer_len = len(spacer)
    off_target_count = 0

    # Check if the genome actually has data
    if len(genome_to_check) < 23:
        print("❌ Error: Genome sequence is too short to scan.")
        return 0

    # Slide across the genome
    for i in range(len(genome_to_check) - (spacer_len + 3)):
        # PAM check (looking for NGG)
        # In a real scan, we check the 3 bases AFTER the 20bp spacer
        pam_site = genome_to_check[i + spacer_len : i + spacer_len + 3]

        if pam_site.endswith("GG"):
            candidate = genome_to_check[i : i + spacer_len]
            mismatches = sum(1 for a, b in zip(spacer, candidate) if a != b)

            # 0-3 mismatches is the 'Danger Zone'
            if mismatches <= 3:
                off_target_count += 1
                print(f"📍 Match at {i} | Seq: {candidate} | PAM: {pam_site} | Mismatches: {mismatches}")

    print(f"--- Scan Finished: {off_target_count} matches found ---")
    return off_target_count

# 2. INTERACTIVE INPUT
print("Example: GACCTTGCGAGAGAAATTAC")
user_spacer = input("Enter your 20nt spacer sequence: ")

# If the user enters nothing, use your gyrA spacer as the default
if not user_spacer:
    user_spacer = "GACCTTGCGAGAGAAATTAC"

# 3. RUN ON TEST DATA (or replace with your loaded genome)
# For this example, we'll create a dummy genome that contains 1 perfect match and 1 off-target
test_genome = ("N" * 50) + user_spacer + "CGG" + ("N" * 20) + "GACCTTGCGAGAGAAATTGC" + "TGG" + ("N" * 50)

scan_off_targets_optimized(user_spacer, test_genome, name="Interactive Test Run")

Example: GACCTTGCGAGAGAAATTAC
Enter your 20nt spacer sequence: GACCTTGCGAGAGAAATTAC

--- 🛡️ Starting Scan: Interactive Test Run ---
Targeting: GACCTTGCGAGAGAAATTAC
📍 Match at 50 | Seq: GACCTTGCGAGAGAAATTAC | PAM: CGG | Mismatches: 0
📍 Match at 93 | Seq: GACCTTGCGAGAGAAATTGC | PAM: TGG | Mismatches: 1
--- Scan Finished: 2 matches found ---


2

In [7]:
# Assuming you ran the !wget and SeqIO steps earlier in the notebook
# This pulls the real DNA sequence from the NCBI file
full_genome_sequence = str(genome_record.seq).upper()

print(f"Total genome size: {len(full_genome_sequence)} bp")

# Run your scanner on the REAL genome
# We use your gyrA spacer: GACCTTGCGAGAGAAATTAC
results_count = scan_off_targets_optimized("GACCTTGCGAGAGAAATTAC", full_genome_sequence, name="E. coli K-12 Genome")

Total genome size: 4641652 bp

--- 🛡️ Starting Scan: E. coli K-12 Genome ---
Targeting: GACCTTGCGAGAGAAATTAC
--- Scan Finished: 0 matches found ---
